In [149]:
import os, io, collections
import pandas as pd
from surprise import Dataset, KNNBaseline, SVD, accuracy, Reader
from surprise.model_selection import cross_validate, train_test_split


# 协同过滤方法

# 载入movielens-100k数据集，一个经典的公开推荐系统数据集，有选项提示是否下载。
# data = Dataset.load_builtin('ml-100k')

# 或载入本地数据集# 数据集路径path to dataset filefile_path = os.path.expanduser('~/.surprise_data/ml-100k/ml-100k/u.data')# 使用Reader指定文本格式，参数line_format指定特征（列名），参数sep指定分隔符reader = Reader(line_format='user item rating timestamp', sep='\t')# 加载数据集data = Dataset.load_from_file(file_path, reader=reader)
file_path = 'ml-100k/ml-100k/u.data'
reader = Reader(line_format='user item rating', sep='\t')
# reader = Reader(rating_scale=(1, 5))
# data = Dataset.load_from_file(file_path, reader=reader)
data3 = pd.read_csv('ml-100k/ml-100k/u.data', sep='\t', header=None, names=['user','item','rating','timestamp'])
data = Dataset.load_from_df(data3[['user','item','rating']],reader=reader)

data_df = pd.read_csv('ml-100k/ml-100k/u.data', sep='\t', header=None, names=['user','item','rating','timestamp'])
item_df = pd.read_csv('ml-100k/ml-100k/u.item', sep='|', encoding='ISO-8859-1', header=None, names=['mid','mtitle']+[x for x in range(22)])
# 每列都转换为字符串类型
data_df = data_df.astype(str)
item_df = item_df.astype(str)
# 电影id到电影标题的映射
item_dict = { item_df.loc[x, 'mid']: item_df.loc[x, 'mtitle'] for x in range(len(item_df)) }

In [85]:
#基于用户的协同过滤算法：
# 使用协同过滤算法时的相似性度量配置
# user-based
user_based_sim_option = {'name': 'pearson_baseline', 'user_based': True}
# item-based
item_based_sim_option = {'name': 'pearson_baseline', 'user_based': False}

# 为用户推荐n部电影，基于用户的协同过滤算法，先获取10个相似度最高的用户，把这些用户评分高的电影加入推荐列表。
def get_similar_users_recommendations(uid, n=10):
    # 获取训练集，这里取数据集全部数据
    trainset = data.build_full_trainset()
#     print(trainset.ur)
    # 考虑基线评级的协同过滤算法
    algo = KNNBaseline(sim_option = user_based_sim_option)
    # 拟合训练集
    algo.fit(trainset)

    # 使用get_neighbors方法得到10个最相似的用户
    neighbors = algo.get_neighbors(int(uid), k=10)
    neighbors_uid = ( algo.trainset.to_raw_uid(x) for x in neighbors )

    recommendations = set()
    #把评分为5的电影加入推荐列表
    for user in neighbors_uid:
        if len(recommendations) > n:
            break
        item = data_df[data_df['user']==str(user)]
        item = item[item['rating']=='5']['item']
        for i in item:
            recommendations.add(item_dict[i])
#     print('\nrecommendations for user %d:')%inner_id
    for i, j in enumerate(list(recommendations)):
        if i >= 10:
            break
        print(j)

In [86]:
#给id为1的用户推荐10部电影：
get_similar_users_recommendations('119', 10)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
False
Air Bud (1997)
That Darn Cat! (1997)
Spawn (1997)
Apt Pupil (1998)
Telling Lies in America (1997)
Lay of the Land, The (1997)
Titanic (1997)
Washington Square (1997)
As Good As It Gets (1997)
Remains of the Day, The (1993)


In [91]:
for i in data_df[data_df['rating']=='5']['item'].values.tolist():
    print(item_dict[i])

Jungle Book, The (1994)
Romy and Michele's High School Reunion (1997)
Star Trek: First Contact (1996)
Age of Innocence, The (1993)
Aladdin (1992)
Jaws (1975)
Rear Window (1954)
Cop Land (1997)
Evil Dead II (1987)
Beautiful Thing (1996)
Last of the Mohicans, The (1992)
Get Shorty (1995)
Return of the Jedi (1983)
Dead Poets Society (1989)
Sound of Music, The (1965)
E.T. the Extra-Terrestrial (1982)
Brothers McMullen, The (1995)
To Kill a Mockingbird (1962)
Fly Away Home (1996)
Star Trek III: The Search for Spock (1984)
Jerry Maguire (1996)
North by Northwest (1959)
Die Hard (1988)
Miller's Crossing (1990)
Monty Python's Life of Brian (1979)
Backbeat (1993)
Adventures of Priscilla, Queen of the Desert, The (1994)
Conspiracy Theory (1997)
Wizard of Oz, The (1939)
Raiders of the Lost Ark (1981)
Terminator 2: Judgment Day (1991)
Willy Wonka and the Chocolate Factory (1971)
Terminator, The (1984)
Shining, The (1980)
Terminator, The (1984)
Amistad (1997)
Taxi Driver (1976)
Tales From the Crypt

Godfather, The (1972)
Lost Horizon (1937)
Four Weddings and a Funeral (1994)
In the Name of the Father (1993)
Dead Man (1995)
Alien (1979)
Mad City (1997)
FairyTale: A True Story (1997)
Blues Brothers, The (1980)
Notorious (1946)
Manhattan (1979)
Terminator 2: Judgment Day (1991)
Apocalypse Now (1979)
Hunt for Red October, The (1990)
Richard III (1995)
Clockwork Orange, A (1971)
When Harry Met Sally... (1989)
Eat Drink Man Woman (1994)
Delta of Venus (1994)
Three Colors: Red (1994)
Amadeus (1984)
Home for the Holidays (1995)
Return of Martin Guerre, The (Retour de Martin Guerre, Le) (1982)
Michael Collins (1996)
Sense and Sensibility (1995)
GoodFellas (1990)
Crossing Guard, The (1995)
Aliens (1986)
Ed Wood (1994)
Star Wars (1977)
Fargo (1996)
Indiana Jones and the Last Crusade (1989)
Empire Strikes Back, The (1980)
Sunset Blvd. (1950)
Leaving Las Vegas (1995)
Koyaanisqatsi (1983)
GoodFellas (1990)
Amateur (1994)
On Golden Pond (1981)
Casablanca (1942)
Chinatown (1974)
Multiplicity (199

Six Degrees of Separation (1993)
Persuasion (1995)
Braveheart (1995)
Home Alone (1990)
Silence of the Lambs, The (1991)
Monty Python and the Holy Grail (1974)
Shawshank Redemption, The (1994)
Boot, Das (1981)
Blade Runner (1982)
Amadeus (1984)
Young Frankenstein (1974)
M*A*S*H (1970)
Good Will Hunting (1997)
Hunt for Red October, The (1990)
Higher Learning (1995)
Phenomenon (1996)
Titanic (1997)
Good Will Hunting (1997)
Sound of Music, The (1965)
Air Force One (1997)
Amadeus (1984)
Pulp Fiction (1994)
Great Escape, The (1963)
Alien (1979)
Cinderella (1950)
When Harry Met Sally... (1989)
Now and Then (1995)
Lone Star (1996)
Right Stuff, The (1983)
Fargo (1996)
Empire Strikes Back, The (1980)
Independence Day (ID4) (1996)
Titanic (1997)
Graduate, The (1967)
Annie Hall (1977)
Marvin's Room (1996)
Fantasia (1940)
Mystery Science Theater 3000: The Movie (1996)
Cool Hand Luke (1967)
Babe (1995)
Ran (1985)
Diva (1981)
Scream 2 (1997)
Rock, The (1996)
Shining, The (1980)
Mary Poppins (1964)
Sw

Die Hard: With a Vengeance (1995)
Primal Fear (1996)
Highlander (1986)
Cat on a Hot Tin Roof (1958)
Casablanca (1942)
Henry V (1989)
Return of the Jedi (1983)
Dead Poets Society (1989)
Swingers (1996)
Hugo Pool (1997)
Air Force One (1997)
Forrest Gump (1994)
Michael (1996)
Rear Window (1954)
Wallace & Gromit: The Best of Aardman Animation (1996)
Bonnie and Clyde (1967)
Forrest Gump (1994)
Return of the Jedi (1983)
Vertigo (1958)
For the Moment (1994)
Amistad (1997)
Paper, The (1994)
When We Were Kings (1996)
Fish Called Wanda, A (1988)
Graduate, The (1967)
Silence of the Lambs, The (1991)
Kull the Conqueror (1997)
Pulp Fiction (1994)
Brazil (1985)
Clockwork Orange, A (1971)
Rumble in the Bronx (1995)
Cry, the Beloved Country (1995)
Grand Day Out, A (1992)
Home Alone (1990)
Beauty and the Beast (1991)
Star Wars (1977)
Terminator, The (1984)
Blue in the Face (1995)
High Noon (1952)
Rainmaker, The (1997)
Top Gun (1986)
Butch Cassidy and the Sundance Kid (1969)
Dead Poets Society (1989)
Bu

Around the World in 80 Days (1956)
Kolya (1996)
Jerry Maguire (1996)
Godfather: Part II, The (1974)
Monty Python's Life of Brian (1979)
Toy Story (1995)
Shining, The (1980)
Brazil (1985)
Anne Frank Remembered (1995)
Forrest Gump (1994)
Lion King, The (1994)
Love Jones (1997)
Dumbo (1941)
Cable Guy, The (1996)
Dumbo (1941)
Wizard of Oz, The (1939)
Seven (Se7en) (1995)
Shall We Dance? (1937)
Full Monty, The (1997)
My Man Godfrey (1936)
Seven (Se7en) (1995)
Snow White and the Seven Dwarfs (1937)
Swingers (1996)
Wrong Trousers, The (1993)
Austin Powers: International Man of Mystery (1997)
Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)
Godfather, The (1972)
Adventures of Priscilla, Queen of the Desert, The (1994)
Wallace & Gromit: The Best of Aardman Animation (1996)
Great Dictator, The (1940)
Hamlet (1996)
Seven (Se7en) (1995)
Smoke (1995)
Blade Runner (1982)
Killing Fields, The (1984)
Lawrence of Arabia (1962)
M*A*S*H (1970)
Fan, The (1996)
To Gillian on Her 37th Birthday (1996)
Whe

...

Die Hard (1988)
2001: A Space Odyssey (1968)
Room with a View, A (1986)
Return of the Jedi (1983)
Beauty and the Beast (1991)
Secret Garden, The (1993)
Game, The (1997)
Titanic (1997)
Apartment, The (1960)
Apt Pupil (1998)
Godfather: Part II, The (1974)
Princess Bride, The (1987)
Kolya (1996)
Terminator, The (1984)
Birds, The (1963)
Forrest Gump (1994)
Freeway (1996)
Star Wars (1977)
Forrest Gump (1994)
Casablanca (1942)
M*A*S*H (1970)
Rosencrantz and Guildenstern Are Dead (1990)
Indiana Jones and the Last Crusade (1989)
Star Wars (1977)
Benny & Joon (1993)
Beauty and the Beast (1991)
Clerks (1994)
Some Like It Hot (1959)
It's a Wonderful Life (1946)
Field of Dreams (1989)
Fifth Element, The (1997)
Clerks (1994)
Beavis and Butt-head Do America (1996)
Postino, Il (1994)
2001: A Space Odyssey (1968)
Aliens (1986)
Lost Horizon (1937)
To Kill a Mockingbird (1962)
Celluloid Closet, The (1995)
Clerks (1994)
Sting, The (1973)
Scream (1996)
Cinema Paradiso (1988)
It's a Wonderful Life (1946)
G

Secrets & Lies (1996)
Terminator 2: Judgment Day (1991)
Willy Wonka and the Chocolate Factory (1971)
Schindler's List (1993)
8 1/2 (1963)
Desperado (1995)
Vertigo (1958)
Dead Poets Society (1989)
Aliens (1986)
Manon of the Spring (Manon des sources) (1986)
Full Monty, The (1997)
Ulee's Gold (1997)
Fargo (1996)
In the Name of the Father (1993)
Hoop Dreams (1994)
Patton (1970)
My Best Friend's Wedding (1997)
Willy Wonka and the Chocolate Factory (1971)
Terminator 2: Judgment Day (1991)
Midnight in the Garden of Good and Evil (1997)
Annie Hall (1977)
Legends of the Fall (1994)
Platoon (1986)
Babe (1995)
Monty Python and the Holy Grail (1974)
Star Wars (1977)
Fargo (1996)
Desperado (1995)
Lone Star (1996)
Singin' in the Rain (1952)
Heathers (1989)
Silence of the Lambs, The (1991)
To Kill a Mockingbird (1962)
One Flew Over the Cuckoo's Nest (1975)
Godfather: Part II, The (1974)
Silence of the Lambs, The (1991)
Godfather: Part II, The (1974)
Apostle, The (1997)
Groundhog Day (1993)
Interview

Eat Drink Man Woman (1994)
Cape Fear (1991)
Mrs. Doubtfire (1993)
Scream (1996)
Thin Man, The (1934)
English Patient, The (1996)
Game, The (1997)
Fargo (1996)
Clear and Present Danger (1994)
Hunt for Red October, The (1990)
Close Shave, A (1995)
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)
Clerks (1994)
Philadelphia (1993)
Bride of Frankenstein (1935)
Princess Bride, The (1987)
Brassed Off (1996)
Toy Story (1995)
Othello (1995)
Amadeus (1984)
English Patient, The (1996)
Bob Roberts (1992)
Foreign Correspondent (1940)
Birds, The (1963)
Terminator, The (1984)
Pulp Fiction (1994)
Richard III (1995)
Butch Cassidy and the Sundance Kid (1969)
Quiet Man, The (1952)
Money Talks (1997)
Gone with the Wind (1939)
Sabrina (1995)
Money Talks (1997)
Three Colors: White (1994)
Rosencrantz and Guildenstern Are Dead (1990)
Contact (1997)
Annie Hall (1977)
English Patient, The (1996)
Nine Months (1995)
Twister (1996)
2001: A Space Odyssey (1968)
Face/Off (1997)
Searching f

Back to the Future (1985)
Around the World in 80 Days (1956)
Fargo (1996)
Return of the Jedi (1983)
Night of the Living Dead (1968)
Raging Bull (1980)
Wrong Trousers, The (1993)
Apt Pupil (1998)
Exotica (1994)
Casablanca (1942)
Graduate, The (1967)
Sting, The (1973)
Ice Storm, The (1997)
Citizen Kane (1941)
Heavenly Creatures (1994)
Cold Comfort Farm (1995)
Jungle Book, The (1994)
Raiders of the Lost Ark (1981)
Star Wars (1977)
Contact (1997)
Titanic (1997)
One Flew Over the Cuckoo's Nest (1975)
Postino, Il (1994)
Dead Man Walking (1995)
Liar Liar (1997)
Rosencrantz and Guildenstern Are Dead (1990)
Return of the Jedi (1983)
Like Water For Chocolate (Como agua para chocolate) (1992)
Thin Man, The (1934)
Terminator, The (1984)
Killing Fields, The (1984)
Star Wars (1977)
Empire Strikes Back, The (1980)
Jackie Chan's First Strike (1996)
From Dusk Till Dawn (1996)
Princess Bride, The (1987)
Godfather: Part II, The (1974)
Scream 2 (1997)
Forrest Gump (1994)
Clerks (1994)
Ransom (1996)
Star W

Philadelphia (1993)
Last of the Mohicans, The (1992)
Shawshank Redemption, The (1994)
Fargo (1996)
Return of the Jedi (1983)
12 Angry Men (1957)
Hunt for Red October, The (1990)
Umbrellas of Cherbourg, The (Parapluies de Cherbourg, Les) (1964)
Faust (1994)
Better Off Dead... (1985)
Life Less Ordinary, A (1997)
Pulp Fiction (1994)
While You Were Sleeping (1995)
Apartment, The (1960)
Dances with Wolves (1990)
North by Northwest (1959)
Lost Highway (1997)
Annie Hall (1977)
Apocalypse Now (1979)
Seventh Seal, The (Sjunde inseglet, Det) (1957)
Welcome To Sarajevo (1997)
African Queen, The (1951)
Indiana Jones and the Last Crusade (1989)
Shawshank Redemption, The (1994)
Terminator, The (1984)
Alien (1979)
Cape Fear (1991)
Indiana Jones and the Last Crusade (1989)
Alice in Wonderland (1951)
Casablanca (1942)
Sting, The (1973)
My Fair Lady (1964)
M*A*S*H (1970)
Heat (1995)
Ice Storm, The (1997)
Silence of the Lambs, The (1991)
Conspiracy Theory (1997)
City of Lost Children, The (1995)
Graduate

In [99]:
import numpy as np
data_df['rating'] = data_df['rating'].astype(int)
grouped = data_df.groupby('item').agg(np.mean)

In [109]:
grouped[grouped.index=='1'].values.tolist()[0][0]

3.8783185840707963

In [121]:
grouped = grouped.sort_values(by='rating',ascending=False)

In [153]:
data_df

,user,item,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156
99999,12,203,3,879959583


In [151]:
s = pd.Series(['1700','242','3','880175444'],index=data_df.columns)
data_df = data_df.append(s,ignore_index=True)

In [152]:
best_item

[('1', 'Toy Story (1995)', 3.8783185840707963),
 ('10', 'Richard III (1995)', 3.831460674157303),
 ('100', 'Fargo (1996)', 4.155511811023622),
 ('1000', 'Lightning Jack (1994)', 3.0),
 ('1001', 'Stupids, The (1996)', 2.0),
 ('1002', 'Pest, The (1997)', 1.875),
 ('1003', 'That Darn Cat! (1997)', 2.25),
 ('1004', 'Geronimo: An American Legend (1993)', 3.111111111111111),
 ('1005',
  'Double vie de Véronique, La (Double Life of Veronique, The) (1991)',
  3.6818181818181817),
 ('1006',
  'Until the End of the World (Bis ans Ende der Welt) (1991)',
  2.8260869565217392),
 ('1007', 'Waiting for Guffman (1996)', 4.127659574468085),
 ('1008', 'I Shot Andy Warhol (1996)', 3.3783783783783785),
 ('1009', 'Stealing Beauty (1996)', 3.375),
 ('101', 'Heavy Metal (1981)', 3.26027397260274),
 ('1010', 'Basquiat (1996)', 3.25),
 ('1011', '2 Days in the Valley (1996)', 3.225806451612903),
 ('1012', 'Private Parts (1997)', 3.53),
 ('1013', 'Anaconda (1997)', 2.289473684210526),
 ('1014', "Romy and Michel

In [160]:
count = 0
for i in item_df.mtitle.values.tolist():
    if i =='Addicted to Love (1997)':
        print(count)
    count+=1

534


In [145]:
1352-1629
993-1255
636-894
287-537

-250